In [28]:
import pandas as pd

train = pd.read_csv("../data/train_processed.csv")
test  = pd.read_csv("../data/test_processed.csv")

print("✅ Datos y preprocesor cargados correctamente.")

✅ Datos y preprocesor cargados correctamente.


In [29]:
# === Cardinalidad de variables categóricas ===

# Identificar columnas categóricas automáticamente
cat_cols = train.select_dtypes(include=["object"]).columns

# Excluir track_id si está presente
cat_cols = cat_cols.drop("track_id")

cardinalities = (
    train[cat_cols]
    .nunique()
    .sort_values(ascending=False)
)

print("Cardinalidad de variables categóricas en TRAIN:")
print(cardinalities)


Cardinalidad de variables categóricas en TRAIN:
track_name     55767
album_name     37315
artists        25775
track_genre      114
dtype: int64


### PASO 1 — Identificar qué variables categóricas tienen alta cardinalidad

Ya identificamos las 3 columnas problemáticas:
- _artists_
- album_name
- track_name

Ahora definimos qué tipo de transformación aplicar a cada una, sabiendo que no podemos hacer One-Hot Encoding por la cantidad enorme de valores distintos.

Para _artists_ vamos a utilizar Count Encoding.  
Para album_name vamos a utilizar Target Encoding (media de popularidad por álbum).  
Para track_name vamos a utilizar Length Encoding.

In [30]:
# === FEATURE 1: Frecuencia del artista (artist_count) ===

# Contar cuántas veces aparece cada artista en Train
artist_counts = train["artists"].value_counts()

# Mapear esa frecuencia a cada fila
train["artist_count"] = train["artists"].map(artist_counts)

# En test usamos los conteos aprendidos en train; artistas nuevos → 0
test["artist_count"] = test["artists"].map(artist_counts).fillna(0)  

# Convertir ambas columnas al mismo tipo
train["artist_count"] = train["artist_count"].astype("int64")
test["artist_count"] = test["artist_count"].astype("int64")

print("\nHead Train artist_count:")
print(train["artist_count"].head())  


print("Head Test artist_count:")
print(test["artist_count"].head())



Head Train artist_count:
0      1
1      7
2     17
3      1
4    117
Name: artist_count, dtype: int64
Head Test artist_count:
0     53
1     12
2      3
3    117
4     51
Name: artist_count, dtype: int64


In [31]:
# === FEATURE 2: Popularidad media por álbum (album_mean_popularity) ===
# Usando Target Encoding con regularización (smoothing) para evitar sobreajuste

# Verificar si album_name existe, si no, recargar datos
if "album_name" not in train.columns:
    print("⚠️  album_name no encontrado. Recargando datos...")
    train = pd.read_csv("../data/train_processed.csv")
    test = pd.read_csv("../data/test_processed.csv")
    print("✅ Datos recargados correctamente.")

target_col = "popularity"

# Calcular la media global de popularidad
global_mean_pop = train[target_col].mean()

# Calcular estadísticas por álbum usando SOLO train
album_stats = train.groupby("album_name")[target_col].agg(["mean", "count"])

# Parámetro de regularización (alpha): mayor valor = más peso a la media global
# Con 65% de álbumes con 1 canción, necesitamos alpha alto para reducir sobreajuste
# alpha=20 reduce significativamente la correlación artificial
alpha = 20  # Ajustable: aumentar si la correlación sigue muy alta

# Target encoding regularizado: combina media del álbum con media global
# Fórmula: (count * mean + alpha * global_mean) / (count + alpha)
# Esto hace que álbumes con pocas canciones se acerquen más a la media global
album_mean_pop = (
    (album_stats["count"] * album_stats["mean"] + alpha * global_mean_pop) 
    / (album_stats["count"] + alpha)
)

# Mapear la media regularizada a cada fila en train
train["album_mean_popularity"] = train["album_name"].map(album_mean_pop)

# Para test: usar la media regularizada del álbum si existe, sino la media global
test["album_mean_popularity"] = (
    test["album_name"].map(album_mean_pop).fillna(global_mean_pop)
)

# Análisis de distribución de álbumes (diagnóstico)
album_counts = train.groupby("album_name").size()
print(f"\n📊 Estadísticas de álbumes:")
print(f"  - Total de álbumes únicos: {len(album_counts)}")
print(f"  - Álbumes con 1 canción: {(album_counts == 1).sum()} ({(album_counts == 1).sum()/len(album_counts)*100:.1f}%)")
print(f"  - Álbumes con ≤3 canciones: {(album_counts <= 3).sum()} ({(album_counts <= 3).sum()/len(album_counts)*100:.1f}%)")
print(f"  - Media de canciones por álbum: {album_counts.mean():.2f}")
print(f"  - Mediana de canciones por álbum: {album_counts.median():.1f}")

print("\nHead Train album_mean_popularity:")
print(train["album_mean_popularity"].head())

print("\nHead Test album_mean_popularity:")
print(test["album_mean_popularity"].head())



📊 Estadísticas de álbumes:
  - Total de álbumes únicos: 37315
  - Álbumes con 1 canción: 24162 (64.8%)
  - Álbumes con ≤3 canciones: 33339 (89.3%)
  - Media de canciones por álbum: 2.14
  - Mediana de canciones por álbum: 1.0

Head Train album_mean_popularity:
0    32.728858
1    32.150273
2    33.604819
3    14.261830
4    31.728313
Name: album_mean_popularity, dtype: float64

Head Test album_mean_popularity:
0    36.937260
1    28.344667
2    31.681239
3     8.213655
4    33.265301
Name: album_mean_popularity, dtype: float64


In [32]:
# === FEATURE 3: track_name ===

def add_track_name_features(df: pd.DataFrame) -> pd.DataFrame:
    # Garantizamos que no haya NaNs
    s = df["track_name"].fillna("")

    # 1. Longitud del nombre (en caracteres)
    df["track_name_len"] = s.str.len()

    # Convertimos a minúsculas para hacer búsqueda segura
    s_lower = s.str.lower()

    # 2. Flag: contiene "remix"
    df["track_has_remix"] = s_lower.str.contains("remix").astype(int)

    # 3. Flag: contiene "live"
    df["track_has_live"] = s_lower.str.contains("live").astype(int)

    # 4. Flag: contiene "acoustic"
    df["track_has_acoustic"] = s_lower.str.contains("acoustic").astype(int)

    return df

# Aplicar a train y test
train = add_track_name_features(train)
test  = add_track_name_features(test)

print(train[[
    "track_name", 
    "track_name_len",
    "track_has_remix",
    "track_has_live",
    "track_has_acoustic"
]].head(20))


                                track_name  track_name_len  track_has_remix  \
0                               Motorcycle              10                0   
1   Addio del passato - From "La traviata"              38                0   
2                       Purple Rose Minuet              18                0   
3             Liebeslied (Widmung), S. 566              28                0   
4                             The Darkside              12                0   
5                                FAKE LOVE               9                0   
6                        Always Be My Baby              17                0   
7                              Finally Out              11                0   
8                                 Limewire               8                0   
9             Chasing Stars - Extended Mix              28                0   
10                After Dark (feat. Fiora)              24                0   
11                                   Solid          

### PASO 2 - Creación de nuevas características

A partir de las variables existentes, crear al menos una nueva característica que aporte valor predictivo al modelo.

Vamos a crear una nueva característica llamada genre_mean_popularity (target encoding de track_genre)

In [33]:
target_col = "popularity"

# media global
global_mean_pop = train[target_col].mean()

# media por género (solo train)
genre_mean_pop = train.groupby("track_genre")[target_col].mean()

# nueva feature en train y test
train["genre_mean_popularity"] = train["track_genre"].map(genre_mean_pop)
test["genre_mean_popularity"] = (
    test["track_genre"].map(genre_mean_pop).fillna(global_mean_pop)
)

# chequeo rápida de valor predictivo
print(train[["genre_mean_popularity", "popularity"]].corr())

                       genre_mean_popularity  popularity
genre_mean_popularity               1.000000    0.503468
popularity                          0.503468    1.000000


### PASO 3 - Eliminar columnas no relevantes

#### Columnas de texto o categóricas

Estas columnas deben eliminarse porque no contienen información útil, tienen cardinalidad absurda, o son directamente identificadores:

1. track_name

2. album_name

3. artists

4. Columnas redundantes después del paso anterior:  Ya creamos genre_mean_popularity, por lo que ya no necesitamos track_genre como categórica. Lo ideal es eliminarla porque:género tiene >100 categorías, y genre_mean_popularity captura mejor la señal.

In [34]:
# Eliminar estas columnas
cols_cat_to_drop = [
    "track_name",
    "album_name",
    "artists",
    "track_genre"
]

train.drop(columns=cols_cat_to_drop, inplace=True, errors="ignore")
test.drop(columns=cols_cat_to_drop, inplace=True, errors="ignore")

print("Columnas categóricas eliminadas:")
print(cols_cat_to_drop)

Columnas categóricas eliminadas:
['track_name', 'album_name', 'artists', 'track_genre']


#### Columnas numéricas

Para evaluar cuáles de estas eliminar, haremos un análisis de correlación con la variable objetivo.

In [35]:
# Identificar columnas numéricas
num_cols_all = train.select_dtypes(include=["number"]).columns

# Calcular correlación con la popularidad
corr_with_popularity = train[num_cols_all].corr()["popularity"].sort_values(ascending=False)

print("📊 Correlación de cada variable numérica con la popularidad:\n")
print(corr_with_popularity)

📊 Correlación de cada variable numérica con la popularidad:

popularity               1.000000
album_mean_popularity    0.785319
genre_mean_popularity    0.503468
loudness                 0.051884
danceability             0.034825
time_signature           0.033124
id                       0.030456
track_has_acoustic       0.017960
tempo                    0.013556
energy                   0.001185
key                     -0.002451
liveness                -0.007030
duration_ms             -0.008599
track_has_live          -0.010305
mode                    -0.015283
acousticness            -0.027883
track_has_remix         -0.034655
valence                 -0.041287
speechiness             -0.045089
track_name_len          -0.072627
artist_count            -0.094448
instrumentalness        -0.095845
Name: popularity, dtype: float64


Se analizó la correlación de cada variable numérica con la popularidad y el rol que cumple dentro del conjunto de datos. 

Vamos a eliminar las columnas: id, key, mode, duration_ms, liveness, acousticness y track_has_live porque presentan una correlación prácticamente nula con la variable objetivo o actuaban como identificadores sin contenido informativo relevante para el modelo.

In [36]:
# Columnas numéricas a eliminar (baja correlación o ruido)
cols_to_drop = [
    "id",
    "key",
    "mode",
    "duration_ms",
    "liveness",
    "acousticness",
    "track_has_live",
]

train.drop(columns=cols_to_drop, inplace=True, errors="ignore")
test.drop(columns=cols_to_drop, inplace=True, errors="ignore")

print("Columnas eliminadas:")
print(cols_to_drop)
print("\nColumnas finales en train:")
print(train.columns)


Columnas eliminadas:
['id', 'key', 'mode', 'duration_ms', 'liveness', 'acousticness', 'track_has_live']

Columnas finales en train:
Index(['track_id', 'popularity', 'explicit', 'danceability', 'energy',
       'loudness', 'speechiness', 'instrumentalness', 'valence', 'tempo',
       'time_signature', 'artist_count', 'album_mean_popularity',
       'track_name_len', 'track_has_remix', 'track_has_acoustic',
       'genre_mean_popularity'],
      dtype='object')


In [37]:
print("Columnas finales del dataset:")
list(train.columns)

Columnas finales del dataset:


['track_id',
 'popularity',
 'explicit',
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'instrumentalness',
 'valence',
 'tempo',
 'time_signature',
 'artist_count',
 'album_mean_popularity',
 'track_name_len',
 'track_has_remix',
 'track_has_acoustic',
 'genre_mean_popularity']

In [38]:
print("Vista previa del dataset transformado:")
train.head()

Vista previa del dataset transformado:


,track_id,popularity,explicit,danceability,energy,loudness,speechiness,instrumentalness,valence,tempo,time_signature,artist_count,album_mean_popularity,track_name_len,track_has_remix,track_has_acoustic,genre_mean_popularity
0,7hUhmkALyQ8SX9mJs5XI3D,22,False,0.305,0.8490,-10.795,0.0549,0.056700,0.3200,141.793,4,1,32.728858,10,0,0,28.882440
1,5x59U89ZnjZXuNAAlc8X1u,22,False,0.287,0.1900,-12.030,0.0370,0.000356,0.1330,83.685,4,7,32.150273,38,0,0,24.859155
2,70Vng5jLzoJLmeLu3ayBQq,37,False,0.583,0.5090,-9.661,0.0362,0.202000,0.5440,90.459,3,17,33.604819,18,0,0,15.750760
3,1cRfzLJapgtwJ61xszs37b,0,False,0.163,0.0368,-23.149,0.0472,0.899000,0.0387,69.442,3,1,14.261830,28,0,0,12.313754
4,47d5lYjbiMy0EdMRV8lRou,27,False,0.647,0.9210,-7.294,0.1850,0.371000,0.1710,137.981,4,117,31.728313,12,0,0,38.806590


In [39]:
print("Dimensiones finales del dataset:", train.shape)

Dimensiones finales del dataset: (79800, 17)


In [40]:
train.to_csv("../data/train_fe.csv", index=False)
test.to_csv("../data/test_fe.csv", index=False)

print("✅ Feature Engineering guardado correctamente.")

✅ Feature Engineering guardado correctamente.
